In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 5.4 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [5]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://juniper-unvoided-wispily.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://juniper-unvoided-wispily.ngrok-free.dev


True

In [6]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        response_modalities=["TEXT"],
    )
)

In [7]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [8]:
result = stateful_query("簡介比特幣")
print(result)

好的，這是一份關於比特幣的簡介：

---

### 比特幣 (Bitcoin, BTC) 簡介

比特幣（Bitcoin, 縮寫 BTC）是**第一個也是最廣為人知**的**去中心化數位加密貨幣**。它是一種**點對點 (peer-to-peer) 的電子現金系統**，允許用戶在**沒有任何中央機構（如銀行或政府）介入**的情況下，直接進行**安全、透明的交易**。

#### 核心特點：

1.  **去中心化 (Decentralization)：** 比特幣最核心的特點。它**不由任何國家、政府或中央銀行發行和控制**。這意味著沒有單一的權力機構可以凍結你的帳戶、控制資金流向或任意增發貨幣。
2.  **區塊鏈技術 (Blockchain Technology)：** 比特幣的**底層技術**。它是一個**公開、透明且不可篡改的「分散式帳本」**。所有比特幣交易都被記錄在這個帳本上，並由網路中的所有參與者共同維護。每個交易被打包成一個「區塊」，鏈接到前一個區塊，形成一條連續的鏈。
3.  **供給有限 (Limited Supply)：** 比特幣的總量是**固定的，最多為 2100 萬枚**。這種稀缺性是其價值主張之一，類似於黃金。新比特幣的產生（稱為「挖礦」）速度會定期減半 (halving)，直到達到總量上限。
4.  **匿名性（或稱假名性）：** 用戶的交易活動與他們的地址（一串加密的數字和字母）相關聯，而不是與他們的真實身份直接相關。雖然交易是公開透明的，但地址背後的人是匿名的。
5.  **挖礦 (Mining)：** 礦工通過解決複雜的數學問題來驗證交易並將其添加到區塊鏈上。成功驗證的礦工會獲得新發行的比特幣和交易手續費作為獎勵。這不僅確保了網路的安全，也是新比特幣產生的方式。

#### 創始人：

比特幣是由一位或一群化名為**中本聰（Satoshi Nakamoto）**的匿名人士於 **2008 年提出概念，2009 年正式上線**。至今，中本聰的真實身份仍是個謎。

#### 主要用途與潛力：

*   **支付手段：** 可以在接受比特幣的商家進行支付。
*   **價值儲存：** 被一些人視為「數位黃金」，用來對抗通膨或作為一種資產儲存。
*   **全球匯款：** 由於其去中心化和相對較低的手續費特性，適合跨境匯款。
*   

In [9]:
result2 = stateful_query("發明者是誰？")
print(result2)

比特幣的發明者是一個化名為 **中本聰（Satoshi Nakamoto）** 的人。

至今，**中本聰的真實身份仍然是個謎**。他可能是一個人，也可能是一個團隊。

中本聰於 **2008 年發布了比特幣的白皮書**，詳細闡述了其概念和運作方式，並於 **2009 年啟動了比特幣網路**。此後不久，他便從公眾視野中消失了，將比特幣的開發工作交給了其他開發者和社區。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Jan/2026 11:40:30] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[]}
BODY:  {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[{"type":"message","message":{"type":"text","id":"594790472968044988","quoteToken":"l4rrzX2upFdwV9qnpjlm0Yg6yk7dk3pRS-YkWaKFLg8Ga2NGf3ZsC1LdhuK88xCuab8CoCAGPLnYZigjv1gldl_7zEBlFAIS9OO8FxM9J38OHjSk9-FWDmqicI8eIqm2kf73q1a3eZckR9haw7w1jQ","markAsReadToken":"aSCfmavmvF08tnBBHvTVSqsa_1PKJ6OLLbFpCDf4EI9nHJqKR5elQjlargZjQ0sTXN08gmX_LWGAuYHQKPvFFFMTJCdI4gnpHfVxzIbNdcIzA2RfSk-ZJsc8bReurMIlyOUAPC3bkq41Rmn2eog4sPMK50CZk1jkISaIGMbYf5SxFU55CkByZpxCiNltv9I_Z-h61hm06VgJod-83OtnGA","text":"AI 簡介比特幣"},"webhookEventId":"01KDZ85CHWNXA8TAT3MKWPVYJW","deliveryContext":{"isRedelivery":false},"timestamp":1767354052671,"source":{"type":"user","userId":"Uafbaa1ab617f08637f77d4fdd09c7975"},"replyToken":"aac4a49e468744dd9e66813a5ef94377","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [02/Jan/2026 11:40:59] "POST / HTTP/1.1" 200 -
